# भाग 2: Syft Keras निर्देशित सुरक्षित मॉडल के साथ काम करना

अब जब आपके पास सामान्य Keras के साथ एक प्रशिक्षित मॉडल है, तो आप कुछ निजी पूर्वानुमान की सेवा के लिए तैयार हैं। हम Syft Keras का उपयोग कर सकते हैं।

इस मॉडल को सुरक्षित करने और सेवा करने के लिए, हमें तीन TFEWorkers (सर्वर) की आवश्यकता होगी। इसका कारण यह है कि TF एनक्रिप्टेड हुड के तहत एक एन्क्रिप्शन तकनीक का उपयोग करता है जिसे [बहु-पक्षीय संगणना (MPC)](https://en.wikipedia.org/wiki/Secure) कहा जाता है। अवधारणा अनुसार, मॉडल वज़न और इनपुट डेटा को शेयरों में विभाजित किया जाता है, फिर प्रत्येक मूल्य का एक हिस्सा अलग-अलग सर्वरों को भेजा जाता है। प्रमुख संपत्ति यह है कि यदि आप एक सर्वर पर शेयर को देखते हैं, तो यह मूल मूल्य (इनपुट डेटा या मॉडल वज़न) के बारे में कुछ नहीं बताता है।

हम एक Syft Keras मॉडल को परिभाषित करेंगे जैसे हमने पिछले नोटबुक में किया था। हालाँकि, एक ट्रिक है: इस मॉडल को इंस्टेंट करने से पहले, हम `hook = sy.KerasHook (tf.keras)` चलाएंगे। यह केरस अनुक्रमिक वर्ग में तीन महत्वपूर्ण नए तरीके जोड़ देगा:
 - `share`: गुप्त साझाकरण के माध्यम से अपने मॉडल को सुरक्षित करेगा; डिफ़ॉल्ट रूप से, यह टीएफ एनक्रिप्टेड से सिक्योरएनएन प्रोटोकॉल का उपयोग करके तीन TFEWorkers में से प्रत्येक के बीच अपने मॉडल को गुप्त रूप से साझा करेगा। सबसे महत्वपूर्ण बात, यह एन्क्रिप्टेड डेटा पर पूर्वानुमान प्रदान करने की क्षमता को जोड़ देगा।
 - `serve`: यह फ़ंक्शन एक सर्विंग कतार लॉन्च करेगा, ताकि TFEWorkers बाहरी ग्राहकों से सुरक्षित मॉडल पर पूर्वानुमान के अनुरोधों को स्वीकार कर सकें।
 - `shutdown_workers`: एक बार जब आप निजी भविष्यवाणियाँ प्रदान करते हैं, तो आप इस फ़ंक्शन को चलाकर अपने मॉडल को बंद कर सकते हैं। यदि आप मैन्युअल रूप से प्रत्येक कार्यकर्ता को प्रबंधित करने का विकल्प चुनते हैं तो यह आपको मैन्युअल रूप से सर्वर प्रक्रियाओं को बंद करने के लिए निर्देशित करेगा।

यदि आप MPC के बारे में अधिक जानना चाहते हैं, तो आप इस उत्कृष्ट [ब्लॉग](https://mortendahl.github.io/2017/04/17/pStreet-deep-learning-with-mpc/) को पढ़ सकते हैं।

अनुवादक - nbTranslate 

संपादक - Urvashi Raheja - Github: [@raheja](https://github.com/raheja)


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

## मॉडल

जैसा कि आप देख सकते हैं, हम लगभग एक ही मॉडल को पहले की तरह परिभाषित करते हैं, सिवाय इसके कि हम एक 'batch_input_shape' प्रदान करते हैं। यह पूर्वनिर्मित टेनसर आकृतियों के माध्यम से सुरक्षित गणनाओं को बेहतर ढंग से अनुकूलित करने के लिए TF एन्क्रिप्टेड को अनुमति देता है। इस MNIST डेमो के लिए, हम (1, 28, 28, 1) के आकार के साथ इनपुट डेटा भेजेंगे।
हम सॉफ्टमैक्स के बजाय लॉगिट को भी लौटाते हैं क्योंकि यह ऑपरेशन एमपीसी का उपयोग करने के लिए जटिल है, और हमें पूर्वानुमान अनुरोधों को पूरा करने के लिए इसकी आवश्यकता नहीं है।

In [2]:
num_classes = 10
input_shape = (1, 28, 28, 1)

In [3]:
model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, name="logit"))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


### पूर्व प्रशिक्षित भार को लोड करें

`load_weights` के साथ आप अपने मॉडल को प्रशिक्षित करने के बाद पहले से बचाए गए भार (weights) को आसानी से लोड कर सकते हैं।

In [4]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

### कार्यकर्ताओं को लॉन्च करें

चलो अब TFEWorkers (`alice`,` bob`, और `carol`) का निर्माण TF Encrypted द्वारा निजी भविष्यवाणियों को करने के लिए आवश्यक है। प्रत्येक TFEWorker के लिए, आपको बस एक होस्ट निर्दिष्ट करना होगा। हम फिर इन श्रमिकों को एक क्लस्टर में जोड़ देते हैं।

ये कार्यकर्ता एक [TensorFlow सर्वर](https://www.tensorflow.org/api_docs/python/tf/distribute/Server) चलाते हैं, जिन्हें आप या तो मैन्युअल रूप से प्रबंधित कर सकते हैं (`AUTO = False`) या श्रमिकों को प्रबंधित करने के लिए कहें। आप (`AUTO = True`)। यदि उन्हें मैन्युअल रूप से प्रबंधित करने के लिए चुना जाता है, तो आपको नीचे दिए गए `cluster.start()` को कॉल करने के बाद प्रत्येक कार्यकर्ता के होस्ट डिवाइस पर एक टर्मिनल कमांड निष्पादित करने का निर्देश दिया जाएगा। यदि सभी श्रमिकों को एक एकल डिवाइस (जैसे `localhost`) पर होस्ट किया जाता है, तो आप कार्यकर्ता के टेन्सरफ्लो सर्वर को स्वचालित रूप से प्रबंधित करने के लिए सिफ्ट का चयन कर सकते हैं।

In [5]:
AUTO = False

alice = sy.TFEWorker(host='localhost:4000', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

cluster = sy.TFECluster(alice, bob, carol)
cluster.start()

INFO:tf_encrypted:If not done already, please launch the following command in a terminal on host localhost:4000: 'python -m tf_encrypted.player --config /var/folders/mh/7hh_sz1d3532_k9w4kcmnt8c0000gn/T/tfe.config server0'
This can be done automatically in a local subprocess by setting `auto_managed=True` when instantiating a TFEWorker.

INFO:tf_encrypted:If not done already, please launch the following command in a terminal on host localhost:4001: 'python -m tf_encrypted.player --config /var/folders/mh/7hh_sz1d3532_k9w4kcmnt8c0000gn/T/tfe.config server1'
This can be done automatically in a local subprocess by setting `auto_managed=True` when instantiating a TFEWorker.

INFO:tf_encrypted:If not done already, please launch the following command in a terminal on host localhost:4002: 'python -m tf_encrypted.player --config /var/folders/mh/7hh_sz1d3532_k9w4kcmnt8c0000gn/T/tfe.config server2'
This can be done automatically in a local subprocess by setting `auto_managed=True` when instantiati

### वजन साझा करके मॉडल को सुरक्षित करें

`sy.KerasHook (tf.keras) के कारण` आप अपने मॉडल को TF एन्क्रिप्ट किए गए Keras मॉडल में बदलने के लिए `share` विधि कह सकते हैं।

यदि आपने ऊपर दिए गए सर्वरों को मैन्युअल रूप से प्रबंधित करने के लिए कहा है, तो यह चरण तब तक पूरा नहीं होगा जब तक कि वे सभी लॉन्च नहीं हो जाते। ध्यान दें कि आपका फ़ायरवॉल आने वाले कनेक्शन को स्वीकार करने के लिए पायथन से पूछ सकता है।

In [6]:
model.share(cluster)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



### Serve model

In [7]:
model.serve(num_requests=3)

Served encrypted prediction 1 to client.
Served encrypted prediction 2 to client.
Served encrypted prediction 3 to client.


आप कुछ निजी भविष्यवाणियों का अनुरोध करने के लिए **भाग 13c** नोटबुक पर जाने के लिए तैयार हैं।

### साफ - सफाई!

एक बार आपके अनुरोध की सीमा ऊपर हो जाने के बाद, मॉडल अब सेवारत अनुरोधों के लिए उपलब्ध नहीं होगा, लेकिन यह अभी भी उपरोक्त तीन श्रमिकों के बीच साझा किया गया रहस्य है। आप नीचे सेल को निष्पादित करके श्रमिकों को मार सकते हैं।

**बधाई** भाग 13 बी खत्म करने पर: सिफ्ट केरस और टीएफई के साथ सुरक्षित वर्गीकरण!

In [8]:
model.stop()
cluster.stop()

if not AUTO:
    process_ids = !ps aux | grep '[p]ython -m tf_encrypted.player --config' | awk '{print $2}'
    for process_id in process_ids:
        !kill {process_id}
        print("Process ID {id} has been killed.".format(id=process_id))

INFO:tf_encrypted:Please terminate the process on host 'localhost:4000'.
INFO:tf_encrypted:Please terminate the process on host 'localhost:4001'.
INFO:tf_encrypted:Please terminate the process on host 'localhost:4002'.


Process ID 12470 has been killed.
Process ID 12474 has been killed.
Process ID 12464 has been killed.
